# User Retention Charts

> This notebook has been published as a live dashboard. 

## 1. Read the data

In [ ]:


sessions_weekly = _deepnote_execute_sql("""select
    sessions.user_id,
    date_trunc('week',users.signed_up_at) as signed_up_at_week,
    floor(extract('day' from session_started_at - signed_up_at)/7) as week -- The number of weeks that passed since the user signed up
from
    sessions
    left join users on sessions.user_id = users.user_id
""", 'SQL_3A66D2DE_5E2B_4CFA_B278_F39246DB3DBA')
sessions_weekly

,user_id,signed_up_at_week,week
0,b2a2a973888c49379fc68dd8193123ec,2021-06-14 00:00:00+00:00,8.0
1,b2a2a973888c49379fc68dd8193123ec,2021-06-14 00:00:00+00:00,14.0
2,b2a2a973888c49379fc68dd8193123ec,2021-06-14 00:00:00+00:00,16.0
3,b2a2a973888c49379fc68dd8193123ec,2021-06-14 00:00:00+00:00,18.0
4,b2a2a973888c49379fc68dd8193123ec,2021-06-14 00:00:00+00:00,18.0
...,...,...,...
190330,731d1e14ed5c4d76a36452c1a8619558,2021-09-27 00:00:00+00:00,0.0
190331,947612f5f28d4190bc9f35e491795839,2021-09-20 00:00:00+00:00,4.0
190332,9d3e70ad52104d6a82b17d4831f3cd7a,2021-06-07 00:00:00+00:00,0.0
190333,9d3e70ad52104d6a82b17d4831f3cd7a,2021-06-07 00:00:00+00:00,1.0


## 2. Select the time range

We have data spanning 365 days, but we don't usually need to look so far back. Let's restrict our analysis to a specific time range.

In [ ]:

from datetime import datetime as _deepnote_datetime
StartDate = _deepnote_datetime.strptime('2021-06-04T00:00:00.000Z', "%Y-%m-%dT%H:%M:%S.%fZ")


In [ ]:

from datetime import datetime as _deepnote_datetime
EndDate = _deepnote_datetime.strptime('2021-09-30T00:00:00.000Z', "%Y-%m-%dT%H:%M:%S.%fZ")


In [ ]:
import pandas as pd

# Round these time ranges to the nearest week
week_start = pd.to_datetime(StartDate).to_period('W').to_timestamp().tz_localize('UTC')
week_end = pd.to_datetime(EndDate).to_period('W').to_timestamp().tz_localize('UTC')

# Filter out any cohorts outside our time range
sessions_weekly = sessions_weekly.loc[
    (sessions_weekly.signed_up_at_week >= week_start) & (sessions_weekly.signed_up_at_week <= week_end)
].copy()

## 3. Calculate retention

We're going to have to transform our data in order to calculate retention. This will involve a few steps:

undefined. Count the number of users in each cohort. That is, the number of users who signed up each week.

undefined. For each week, count the number of users from each cohort who were active that week.

undefined. Divide the number of users active that week by the total size of the cohort. This will give us the retention rate.

In [ ]:
retention = sessions_weekly.copy()

# Save the cohort size before we start calculating retention
cohort_size = retention.groupby(['signed_up_at_week']).user_id.nunique().reset_index(name='cohort_size')

# For each cohort-week, calculate the number of users who visited
retention = retention.groupby(['signed_up_at_week','week']).user_id.nunique().reset_index(name='n_users')

# Pivot and melt the table. This is a little trick that allows us to add rows during weeks where a cohort was not active.
retention = retention.pivot(index=['signed_up_at_week'],columns='week',values='n_users').fillna(0)
retention = retention.melt(value_name='n_users',ignore_index=False).reset_index()

# If part of the cohort is still not finished the week, then exclude that cohort-week from the data
# To do this we add 6 days to the sign up date. This gives us the last sign ups of that cohort.
retention = retention.loc[
    ~(retention.signed_up_at_week + pd.to_timedelta(retention.week + 1,'W') + pd.Timedelta(6,'D') >
    pd.Timestamp.now(tz='UTC').floor('D'))
]

# Divide by the cohort size to get a percentage
retention = retention.merge(cohort_size,on=['signed_up_at_week'])
retention['prop'] = retention.n_users / retention.cohort_size

## 4. Visualize

In [ ]:
# Rename the columns so they are more pretty
retention = retention.rename(columns = dict(
    prop='Retention',
    week='Week',
    signed_up_at_week='Cohort',
    n_users='Number of users',
    cohort_size='Cohort size'
))

Let's start by plotting a time series of each cohort's retention rate. This is useful for identifying changes over time. For example, we can see that our cohorts in September are performing much better than the ones in May through August.

In [ ]:
DeepnoteChart(retention, """{"data":{"name":"placeholder"},"mark":{"type":"line","tooltip":{"content":"data"}},"height":220,"$schema":"https://vega.github.io/schema/vega-lite/v4.json","autosize":{"type":"fit"},"encoding":{"x":{"sort":null,"type":"ordinal","field":"Week","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Retention","scale":{"type":"linear","zero":false}},"color":{"sort":null,"type":"ordinal","field":"Cohort","scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

Sometimes it's helpful to look at the number of users, not just the retention rate. It's also useful to visualize retention as a matrix, to find any outliers. Once again, we can see our September cohorts is doing well.

In [ ]:
import altair as alt

base=alt.Chart().encode(
    x=alt.X('Week:N', title='Week', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('monthdate(Cohort):N', title='Cohort')
)
rect=base.mark_rect(tooltip=True).encode(
    alt.Color('Retention', scale=alt.Scale(scheme='greenblue'))
)

text=base.mark_text(tooltip=True, color='white').encode(
    alt.Text('Number of users', format=''), 
)

(rect+text).properties(data=retention, width=800, height=500)

alt.LayerChart(...)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=19b741c6-0737-41ef-99b4-832fc7c86171' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>